In [2]:
# https://stackoverflow.com/questions/66951471/modulenotfounderror-no-module-named-indexer
!pip install pyspellchecker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 35.1 MB/s eta 0:00:00


In [3]:
import requests
from transformers import pipeline
from spellchecker import SpellChecker

In [58]:
from transformers import pipeline
fill_mask = pipeline("fill-mask", model='bert-base-uncased', top_k=100)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issu

In [59]:
# Function to get homophones from Datamuse API
def get_homophones(word):
    response = requests.get(f"https://api.datamuse.com/words?rel_hom={word}")
    homophones = [entry['word'] for entry in response.json()]
    return homophones

In [60]:
# Fixing masking tokens in other words i.e masked token word a appears in taco
def wordsOnly(text):

  text_words = text.split()

  text_words_only = []

  for word in text_words:

    # https://stackoverflow.com/questions/704152/how-can-i-convert-a-character-to-a-integer-in-python-and-viceversa
    # ascii values from
    # https://www.geeksforgeeks.org/ascii-table/
    if ((ord(word[-1]) < 65) or (ord(word[-1]) > 90) and (ord(word[-1]) < 97)) or (ord(word[-1]) > 122):

      word = word[:-1]

    text_words_only.append(word)

  return text_words_only

In [105]:
# Fixing masking tokens in other words i.e masked token word a appears in taco
def wordCheck(text):

  text = text.lower()

  # Extract each word. Remove punctuation at the end
  testing_words = wordsOnly(text)

  spell = SpellChecker()

  # find those words that may be misspelled
  misspelled = spell.unknown(testing_words)

  #print(misspelled)

  # This section modifies what we are looking for in the text addressing
  # replacing words like "a" which are found inside "taco"

  for word in misspelled:

    candidates = spell.candidates(word)

    #print(candidates)

    if (len(testing_words) >= 2):

      if (word == testing_words[0]):

        # add space after word
        word_sp = word + " "

        # add space after mask
        mask = "[MASK] "

      elif (word == testing_words[-1]):

        # add space after word
        word_sp = " " + word

        # add space after mask
        mask = " [MASK]"

      else:

        # add space after word
        word_sp = " " + word + " "

        # add space after mask
        mask = " [MASK] "

    else:

      word_sp = word
      mask = "[MASK]"


    # Replace word with a mask for BERT
    masked_sentence = text.replace(word_sp, mask)

    # Apply BERT to predict the best word
    predictions = fill_mask(masked_sentence)

    for prediction in predictions:

      #print("Prediction: ", prediction)

      if (prediction['token_str'] in candidates):

        text = masked_sentence.replace("[MASK]", prediction['token_str'])

        #print("Bert selected: ", prediction['token_str'])

        break

    #print("\n")

  return text.upper()

In [113]:
# Fixing masking tokens in other words i.e masked token word a appears in taco
def homonymCheck(text):

  text = text.lower()

  # Extract each word. Remove punctuation at the end
  testing_words = wordsOnly(text)

  # This section modifies what we are looking for in the text addressing
  # replacing words like "a" which are found inside "taco"
  for word in testing_words:

    if (len(testing_words) >= 2):

      if(word == testing_words[0]):

        # add space after word
        word_sp = word + " "

        # add space after mask
        mask = "[MASK] "

      elif (word == testing_words[-1]):

        # add space after word
        word_sp = " " + word

        # add space after mask
        mask = " [MASK]"

      else:

        # add space after word
        word_sp = " " + word + " "

        # add space after mask
        mask = " [MASK] "

    else:

        word_sp = word
        mask = "[MASK]"

    # Replace word with a mask for BERT
    masked_sentence = text.replace(word_sp, mask, 1)

    # Apply BERT to predict the best word
    predictions = fill_mask(masked_sentence)

    #words = [word]

    #print("Original word: ", word)

    homophones = get_homophones(word)
    #synonyms = get_synonyms(word)
    #synonyms_1 = get_synonyms(word[:-1])

    #print("Homophones: ", homophones)
    #print("Synonyms: ", synonyms)
    #print("Synonyms -1: ", synonyms_1)

    #print(predictions)

    for prediction in predictions:

      #print("Prediction: ", prediction)

      if (prediction['token_str'] == word) or (prediction['token_str'] in homophones):

        text = masked_sentence.replace("[MASK]", prediction['token_str'])

        #print("Bert selected: ", prediction['token_str'])

        break

    #print("\n")

  return text.upper()


In [114]:
def spellCheck(text):
  correct = wordCheck(text)
  return homonymCheck(correct)

In [115]:
# YOLO OUTPUTs

# CONFUSING WORDS ANALYSIS

# Many issues come from a poor diet. For example, sweets can Xffect your health.
# Effect x Affect example

input_1 = "MANY ISSUES COME FROM A POOR DIET FOR EXAMPLE SWEETS CAN XFFECT YOUR HEALTH"

# The PRINCIPLE of the school is very strict.
# principle x principal example

input_2 = "THE PRINCIPEL OF THE SCHOOL IS VERY STRICT"

# You should know how to SITE your sources
# cite x site x sight example

input_3 = "YOU NEED TO SITE YOUR REFERENCES CORRECTLY IN YOUR RESEARCH PAPER."

# After a big family dinner, everyone was excited to see what delicious DESERT would be served.
# dessert x desert example

input_4 = "AFTER A BIG FAMILY DINNER EVERYONE WAS EXCITED TO SEE WHAT DELICIOUS DESERT WOULD BE SERVED"

# FILL THE WORD ANALYSIS

# Context is important for understanding the meanings.
# Example with a missing letter in "understanding"

input_5 = "CONTEXT IS IMPORTANT FOR UNDERSANDING THE MEANINGS"

# Traveling exposes you to new cultures and experiences.
# Example with a missing letter in "cultures."

input_6 = "TRAVELING EXPOSES YOU TO NEW CULTURS AND EXPERIENCES"

# Technology bridges gaps between distant communities.
# Example with missing letters in "communities."

input_7 = "TECHNOLOGY BRIDGES GAPS BETWEEN DISTANT COMUNITIES"

# COMMON WORDS MISPELLED

# Reading regularly can improve cognitive skills.
# Misspelled word: "cognitive" -> "cognitiva"

input_8 = "READING REGULARLY CAN IMPROVE COGNITIVA SKILLS"

# The experiment showed good results.
# Misspelled word: "experiment" -> "experilent"

input_9 = "THE EXPERILENT SHOWED GOOD RESULTS"

# Physical activity boosts energy levels.
# Misspelled word: "activity" -> "aktivwty"

input_10 = "PHYSICAL AKTIVWTY BOOSTS ENERGY LEVELS"


In [116]:
spellCheck("a")

'A'

In [117]:
wordCheck("a")

'A'

In [118]:
wordCheck("")

''

In [119]:
spellCheck(input_10)

'PHYSICAL ACTIVITY BOOSTS ENERGY LEVELS'

In [120]:
spellCheck(input_9)

'THE EXPERIMENT SHOWED GOOD RESULTS'

In [121]:
spellCheck(input_8)

'READING REGULARLY CAN IMPROVE COGNITIVE SKILLS'

In [122]:
spellCheck(input_7)

'TECHNOLOGY BRIDGES GAPS BETWEEN DISTANT COMMUNITIES'

In [123]:
spellCheck(input_6)

'TRAVELING EXPOSES YOU TO NEW CULTURES AND EXPERIENCES'

In [124]:
spellCheck(input_5)

'CONTEXT IS IMPORTANT FOR UNDERSTANDING THE MEANINGS'

In [125]:
spellCheck(input_4)

'AFTER A BIG FAMILY DINNER EVERYONE WAS EXCITED TO SEE WHAT DELICIOUS DESSERT WOULD BE SERVED'

In [126]:
spellCheck(input_3)

'YOU NEED TO CITE YOUR REFERENCES CORRECTLY IN YOUR RESEARCH PAPER.'

In [127]:
spellCheck(input_2)

'THE PRINCIPAL OF THE SCHOOL IS VERY STRICT'

In [128]:
spellCheck(input_1)

'MANY ISSUES COME FROM A POOR DIET FOR EXAMPLE SWEETS CAN AFFECT YOUR HEALTH'

In [129]:
while 1:

  user_input = input("Type a sentence: ")

  if user_input == "exit":
    break
  else:
    print(spellCheck(user_input))

Type a sentence: MANY ISSUES COME FROM A POOR DIET FOR EXAMPLE SWEETS CAN AFFECT YOUR HEALTH
MANY ISSUES COME FROM A POOR DIET FOR EXAMPLE SWEETS CAN AFFECT YOUR HEALTH
Type a sentence: MANY ISSUES COME FROM A POOR DIET FOR EXAMPLE SWEETS CAN XFFECT YOUR HEALTH
MANY ISSUES COME FROM A POOR DIET FOR EXAMPLE SWEETS CAN AFFECT YOUR HEALTH
Type a sentence: exit


In [130]:
spellCheck("realy")

'REALY'